In [ ]:
import nltk
import string
from nltk.util import ngrams
from collections import Counter
from nltk.tokenize import TweetTokenizer
import string
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def removePunctuations(sen):
    temp_l = sen.split()
    i = 0
    j = 0
    contrac = ['s','t','d','ve','re','ll']
    for word in temp_l :
        j = 0
        for l in word :
            if l in string.punctuation:
                if l == "'":
                    if j+1<len(word) and word[j+1] in contrac:
                        j = j + 1
                        continue
                word = word.replace(l," ")
            j += 1
        temp_l[i] = word.lower()
        i=i+1   
    content = " ".join(temp_l)
    return content

In [ ]:
def get_ngrams(sentences, n ):
    sents = []
    tknzr = TweetTokenizer()
    for sent in sentences:
        n_grams = ngrams(tknzr.tokenize(removePunctuations(sent)), n)
        sents.extend(n_grams)
    return sents

In [ ]:
sentences = []
with open("/content/text.txt",'r',encoding = 'utf-8') as f:
    sentences = f.readlines()
print(len(sentences))
print(sentences[0:5])

15110
['How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\n', "When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\n", "they've decided its more fun if I don't.\n", 'So Tired D; Played Lazer Tag & Ran A LOT D; Ughh Going To Sleep Like In 5 Minutes ;)\n', 'Words from a complete stranger! Made my birthday even better :)\n']


In [ ]:
sent_tri_grams = get_ngrams(sentences,3)
print(sent_tri_grams[0:5])
freq_tri = nltk.FreqDist(sent_tri_grams)
print("\nMost common trigrams: ", freq_tri.most_common(10))
counter_trigram = Counter(sent_tri_grams)
print('Length counter tri-grams: ',len(counter_trigram))

[('how', 'are', 'you'), ('are', 'you', 'btw'), ('you', 'btw', 'thanks'), ('btw', 'thanks', 'for'), ('thanks', 'for', 'the')]

Most common trigrams:  [(('thanks', 'for', 'the'), 157), (('thank', 'you', 'for'), 62), (('i', 'love', 'you'), 60), (('i', 'm', 'not'), 55), (('i', 'want', 'to'), 53), (('looking', 'forward', 'to'), 52), (('going', 'to', 'be'), 51), (("can't", 'wait', 'to'), 51), (('for', 'the', 'follow'), 48), (('and', 'i', 'm'), 38)]
Length counter tri-grams:  147475


In [ ]:
tri_dict={}
set_tri_grams = set(sent_tri_grams)
for gram in set_tri_grams:
    key=(gram[0],gram[1])
    if key in tri_dict.keys():
        tri_dict[key].append(gram[2])
    else:
        tri_dict[key]=[gram[2]]
print(len(tri_dict))
print(list(tri_dict.keys())[0:10])
print('day off: ?')
print(tri_dict[('day','off')])

94598
[('tampa', 'bay'), ('bring', 'me'), ('the', 'follow'), ('1', '2'), ('up', 'and'), ('cont', 'visit'), ('well', 'igotta'), ('little', 'later'), ('been', 'shown'), ('barely', 'got')]
day off: ?
['we', 'drank', 'since', 'in', 'from', 'tomorrow', 'what']


In [ ]:
def findWordNext(tokens, ngram_dict):
    i=len(tokens)-1
    key=(tokens[i-1],tokens[i])
    dict_nextword={}
    if key in ngram_dict.keys():
        for next_word in ngram_dict[key]:
            dict_nextword[next_word]=counter_trigram[(key[0],key[1],next_word)]
        sorted_dict=sorted(dict_nextword,key=dict_nextword.__getitem__,reverse=True)
    return sorted_dict

In [ ]:
sent_temp = "The tragedy of life is not that"
tknzr = TweetTokenizer()
token_temp = tknzr.tokenize(removePunctuations(sent_temp))
print(findWordNext(token_temp, tri_dict))

['hard', 'it', 'scary', 'the', 'anyone', 'i', 'players', 'makes', 'anybody', 'cute', 'girl', 'long', 'pug', 'surprising']


In [ ]:
def findNext2W(tokens1, tokens2, list_fw, ngram_dict=tri_dict, func_find_word=findWordNext):
    temp = True
    count = 0
    while temp:
        tokens1.append(list_fw[count])
        fwn = func_find_word(tokens1, ngram_dict)
        if tokens2[0] not in fwn:
            tokens1.pop()
            count += 1
        else:
            for i in fwn:
                if i == tokens2[0]:
                    tokens1.append(i)
                    fwn = func_find_word(tokens1, ngram_dict)
                    if  tokens2[1] not in fwn:
                        tokens1 = tokens1[:len(tokens1)-2]
                        count += 1
                        continue
                    else:
                        for j in fwn:
                            if j == tokens2[1]:
                                tokens1.append(j)
                        temp = False
    return tokens1

In [ ]:
def createTokens(seq):
    sen1 ,sen2 = seq.split("[X]")[0], seq.split("[X]")[1]
    tknzr = TweetTokenizer()
    tokens_1 = tknzr.tokenize(removePunctuations(sen1))
    tokens_2 = tknzr.tokenize(removePunctuations(sen2))
    return tokens_1, tokens_2

In [ ]:
def wordMissing(tokens1, tokens2):
    list_fwn = findWordNext(tokens_1,tri_dict)
    l = findNext2W(tokens1, tokens2,list_fwn)
    if l[len(l)-2]== tokens2[0] and l[len(l)-1]== tokens2[1]:
        word_missing = l[len(l)-3]
    return word_missing

In [ ]:
sentence_complete = "Your heart will beat more rapidly and you'll smile for no reason"
sent_miss = "Your heart will beat more rapidly [X] you'll smile for no reason"
tokens_1, tokens_2 = createTokens(sent_miss)
print("[X]:", wordMissing(tokens_1,tokens_2))

[X]: and


In [ ]:
test_data = sentences[0:2000]
data_temp = [i.split() for i in test_data]
x = []
data_x = []
for d in data_temp:
    x.append(d[len(d)//2])
    d[len(d)//2]="[X]"
    data_x.append(d)

sent_test = [" ".join(i) for i in data_x]
x =list(map(lambda x:x.lower(), x))
x = [''.join(c for c in s if c not in string.punctuation) for s in x]
print(sent_test)
print(len(sent_test))

['How are you? Btw thanks for the RT. You gonna be in [X] anytime soon? Love to see you. Been way, way too long.', "When you meet someone special... you'll know. Your heart [X] beat more rapidly and you'll smile for no reason.", "they've decided its more [X] if I don't.", 'So Tired D; Played Lazer Tag & Ran A LOT [X] Ughh Going To Sleep Like In 5 Minutes ;)', 'Words from a complete stranger! [X] my birthday even better :)', 'First Cubs game ever! Wrigley field is [X] This is perfect. Go Cubs Go!', 'i no! i get another day off from skool due [X] the wonderful snow (: and THIS wakes me up...damn thing', "I'm coo... Jus at work hella [X] r u ever in cali", 'The new sundrop commercial [X] love at first sight', 'we need to [X] THIS WEEK', 'I always wonder how the guys on the auctions shows [X] to talk so fast!? all I hear is djsosnekspqnslanskam.', 'Dammnnnnn what [X] catch', 'such a great picture! The green [X] totally brings out your eyes!', 'Desk put together, room all [X] up. Oh boy, oh

In [ ]:
word_missing_predict = []
sent_err = []
count_err = 0
for sent in sent_test:
    tokens_1, tokens_2 = createTokens(sent)
    try:
        X = wordMissing(tokens_1,tokens_2)
    except:
        sent_err.append(sent)
        word_missing_predict.append(0)
        count_err += 1
    else:
        word_missing_predict.append(X)

In [ ]:
print(len(word_missing_predict))
print(count_err)
print(sent_err)

2000
439
['Dammnnnnn what [X] catch', 'ford [X] hatchback?', 'RT : According to the National Retail Federation [X] BILLION was spent on #MothersDay last year!!', 'watch your [X] : )', 'Tommorows [X] day...', 'Bum Squad lets get it! RT [X] Shout that ninja out for winning', 'Exhaust [X] arrrgh', 'i Love Reading your magazine [X] it always cheers me up', 'I [X] <3', 'Sing [X]', 'GOP line on Obama gay marriage stance seems to be that he [X] Really want to use that with Romney as your presidential candidate?', 'No stress balls or keg and different crew [X] but these guys know how to party', 'got a [X] one?', 'love [X] brown', 'its [X] good', 'How (un??)fortunate that those days when every status update was a [X] lyrics were also the days when I first discovered New Wave...', "I had a bomb [X] day Chillin' with friends.", 'Enjoy!! [X] cool!!', 'talks in [X] person.', "I'm taking [X] :-)", 'lets [X] this', "Not much, Ben [X] how's you?", 'Time [X] lunch!', 'at [X] flags.', 'Thanks [X]', "thi

In [ ]:
print(x)
print(word_missing_predict)

['dc', 'will', 'fun', 'd', 'made', 'gorgeous', 'to', 'tired', 'hehe', 'reconnect', 'learned', 'a', 'shirt', 'set', 'doing', 'office', 'blog', 'street', 'focus', '163', 'but', 'dinner', 'mailbox', 'the', 'on', 'lot', 'it', 'for', '', 'stairs', 'right', 'leak', '', 'for', 'better', 'mentioned', 'will', 'forward', 'you', 'it', 'tonight', 'over', 'good', 'we', 'for', 'and', 'flipflopped', 'goes', '', 'he', 'i', 'the', 'good', 'chris', 'sexy', 'sounds', 'song', 'ass', 'open', 'by', 'who', 'going', 'about', 'rt', 'stay', 'know', 'third', 'control', 'do', 'that', 'the', 'adam', 'do', '', 'my', 'only', 'be', 'rt', 'for', 'anyone', 'with', 'its', 'it', 'a', 'playoffs', 'can', 'me', 'be', 'therave', 'six', 'tom', 'you', 'that', 'majors', 'at', 'wordpress', 'through', 'a', 'needs', 'type', 'made', 'dry', 'tweet', 'the', 'is', 'was', 'a', 'is', 'in', 'voicemails', 'better', 'with', 'that', 'is', 'a', 'months', 'exceptional', 'like', 'too', 'following', 'minutes', 'and', 'the', 'favorite', 'do', 'j

In [ ]:
c = 0
for i in range(len(x)):
    if x[i] == word_missing_predict[i]:
        c += 1
print(f"test accuracy: { round(c/len(x) * 100, 2)}%")

test accuracy: 75.55%
